### Imports

In [3]:
from os.path import join

from keras.applications import VGG16, VGG19, InceptionV3, Xception, ResNet50
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model, load_model
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, Callback
from keras.utils.training_utils import multi_gpu_model
from keras.optimizers import Adam

import tensorflow as tf

import os
import numpy as np

from batch_generator import BatchGenerator, BatchSequence

from sklearn.metrics import recall_score, precision_score, f1_score

import gzip, pickle

### Load data

In [4]:
# Images
images_path_train = os.path.abspath('data/train/')
images_path_validation = os.path.abspath('data/validation/')
images_path_test = os.path.abspath('data/test/')

# Labels
with gzip.open('data/y_train.pickle','rb') as fp:
    y_train = pickle.load(fp)
with gzip.open('data/y_validation.pickle','rb') as fp:
    y_validation = pickle.load(fp)

### Metrics / callbacks

In [5]:
class Metrics(Callback):

    def on_train_begin(self, logs={}):
        self.mean_f1s = []
        self.recalls = []
        self.precisions = []

    def on_epoch_end(self, epoch, logs={}):
        y_pred = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        y_true = self.validation_data[1]

        mean_f1 = f1_score(y_true, y_pred, average='micro')
        recall = recall_score(y_true, y_pred, average='micro')
        precision = precision_score(y_true, y_pred, average='micro')
        self.mean_f1s.append(mean_f1)
        self.recalls.append(recall)
        self.precisions.append(precision)

        print('mean_F1: {} — precision: {} — recall: {}'.format(mean_f1, precision, recall))

metrics = Metrics()

In [ ]:
# checkpoint
filepath="models/inceptionV3-fc-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

## Network

In [3]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
batch_size=128*2 # 128 per GPU

In [5]:
# load the generators
training_gen = BatchGenerator(input_dir=images_path_train, y=y_train, batch_size=batch_size)

In [6]:
val_gen = BatchSequence(input_dir=images_path_validation, y=y_validation, batch_size=batch_size)

model       trainable layers
vgg16       4
vgg19       5
exception   6
inception   17
resnet50    7

#### Function that takes modelpath and number of epochs and trains the model at the specified path for the number of epochs. After each epoch the model is saved.

In [8]:
def train_network(modelname, epochs):    
    with tf.device('/cpu:0'):
        model = load_model('models/{}-finetuned-3500_steps.h5'.format(modelname)) 
        #models/{}-fc-1_epochs.h5
        trainable_layers = layer_dict[modelname[:5]]
        for layer in model.layers[:trainable_layers]:
            layer.trainable = False
        for layer in model.layers[trainable_layers:]:
            layer.trainable = True

    # Multi-GPU data parallelism
    multi_model = multi_gpu_model(model, gpus=4)

    # Use binary loss instead of categorical loss to penalize each output independently, also use lower learning rate
    optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    multi_model.compile(optimizer=optimizer, loss='binary_crossentropy')

    steps = int(y_train.shape[0]/batch_size) + 1
    steps = 250

    for i in range(14, epochs):
        history = multi_model.fit_generator(training_gen, steps_per_epoch=steps, epochs=1)
        model.save( "models/{}-finetuned-{}_steps.h5".format(modelname, (i+1)*steps) )

#### Train all models, if you want to train only 1 model then remove the loop :P

In [9]:
models = ['Xception', 'VGG16', 'VGG19', 'ResNet50']
layer_dict = {'VGG16':-8, 'VGG19':-9, 'Xcept':-10, 'ResNe':-11, 'Incep':-21}
epochs = 16

train_network('VGG19', epochs)

/root/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Epoch 1/1
250/250 [==============================] - 1084s 4s/step - loss: 0.0556
Epoch 1/1
250/250 [==============================] - 1073s 4s/step - loss: 0.0553


### Test models


In [2]:
predict_gen = BatchSequence(input_dir=images_path_validation, y=y_validation, batch_size=128)

for path in ['models/Xception-finetuned-4000_steps.h5','models/ResNet50-finetuned-1_epochs.h5', 'models/VGG19-finetuned-1_epochs.h5', 
             'models/Xception-finetuned-1_epochs.h5', 'models/VGG16-finetuned-1_epochs.h5']:
    model = load_model(path)
    # Train only the top few layers
        
    optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer=optimizer, loss='binary_crossentropy') # training configuration
    
    predictions = model.predict_generator(predict_gen, verbose=1)

    y_true = y_validation
    y_pred = (predictions > 0.5).astype(int)

    pr = precision_score(y_true, y_pred, average='micro')
    rc = recall_score(y_true, y_pred, average='micro')
    f1 = f1_score(y_true, y_pred, average='micro')

    print("[{}] Precision: {} Recall: {} F1: {}".format(path, pr, rc, f1))
    break

NameError: name 'images_path_validation' is not defined

In [ ]:
[models/VGG19-finetuned-4000_steps.h5] Precision: 0.8499547951874261 Recall: 0.30783859348631015 F1: 0.45197855029585804
[models/Xception-finetuned-4000_steps.h5] Precision: 0.4570333880678708 Recall: 0.14722551069242587 F1: 0.22270908744522763